# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
ndf = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = ndf['Survived']
ndf = ndf.drop(['PassengerId', 'Survived'] , axis=1)
ndf.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
num_features = []
for dtype, feature in zip(ndf.dtypes, ndf.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')
opj_features = ndf.columns.drop(num_features)
opj_features

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [30]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
n = 'Age'
cl = 'Sex'
for num in num_features[:]:
    for cl in opj_features[:]:
        if(num=='Age' and (cl=='Name' or cl=='Ticket'or cl=='Cabin')):
            continue
        
        df = pd.DataFrame.copy(ndf)
        df[cl] = df[cl].fillna('None')
        mean_df = df.groupby([cl])[num].mean().reset_index()
        mode_df = df.groupby([cl])[num].apply(lambda x: x.mode()[0]).reset_index()
        median_df = df.groupby([cl])[num].median().reset_index()
        max_df = df.groupby([cl])[num].max().reset_index()
        temp = pd.merge(mean_df, mode_df, how='left', on=[cl])
        temp = pd.merge(temp, median_df, how='left', on=[cl])
        temp = pd.merge(temp, max_df, how='left', on=[cl])
        columnsname = [cl, cl+'_Mean',  cl+'_Mode',  cl+'_Median',  cl+'_Max']
        temp.columns = columnsname


        df = pd.merge(df,temp,how="left",on=[cl])


        # 削減文字型欄位, 只剩數值型欄位
        df = df.drop(opj_features,axis=1)
        df = df.fillna(-1)
        MMEncoder = MinMaxScaler()


        # 原始特徵 + 邏輯斯迴歸
        odf = df.drop(columnsname[1:],axis=1)

        train_X = MMEncoder.fit_transform(odf)
        estimator = LogisticRegression()
        old = cross_val_score(estimator, train_X, train_Y, cv=5).mean()


        # 新特徵 + 邏輯斯迴歸
        train_X = MMEncoder.fit_transform(df)
        estimator = LogisticRegression()
        new = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
        
        # 新特徵 > 原始特徵
        if new-old>0:
            print(f"{num}\t{cl}\t\t{old:.6f}\t{new:.6f}")

Pclass	Sex		0.703864	0.783414
Age	Sex		0.703864	0.783407
SibSp	Sex		0.703864	0.782277
Parch	Sex		0.703864	0.782271
Parch	Ticket		0.703864	0.708377
Fare	Sex		0.703864	0.782277
